John Shamshoian

Homework 4

## Q1

For a multivariate count vector $\mathbf{x}=(x_1,\ldots,x_d)$ with batch size $|\mathbf{x}|=\sum_{j=1}^d x_j$, show that the probability mass function for Dirichlet-multinomial distribution is
$$
    f(\mathbf{x} \mid \alpha) 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}  
    = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$
where $\Delta_d$ is the unit simplex in $d$ dimensions and $|\alpha| = \sum_{j=1}^d \alpha_j$.


If $\mathbf{p} \sim \text{Dir}(\mathbf{\alpha})$, $\mathbf{p}$ is a probability density. Therefore 

$$
\int_{\Delta_d}\pi(\mathbf{p})\,d\mathbf{p} =  \int_{\Delta_d}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)}\prod_{j=1}^d p_j^{\alpha_j-1} \, d\mathbf{p} = 1
$$

Rearranging, I get $$\int_{\Delta_d}\prod_{j=1}^d p_j^{\alpha_j-1}\,d\mathbf{p} = \frac{\prod_{j=1}^d \Gamma(\alpha_j)}{\Gamma(|\alpha|)}$$

If $\mathbf{x} \sim \text{Multinomial}(\mathbf{p})$ and $\mathbf{p} \sim \text{Dir}(\mathbf{\alpha})$, then $p(\mathbf{x}\,|\,\mathbf{p},\mathbf{\alpha}) = p(\mathbf{x}\,|\,\mathbf{p})$. Using this fact, I have

$$
\begin{align}
p(\mathbf{x}, \mathbf{p}\,|\,\mathbf{\alpha})&=\frac{p(\mathbf{x},\mathbf{p},\mathbf{\alpha})}{p(\mathbf{\alpha})}\cdot\frac{p(\mathbf{p},\mathbf{\alpha})}{p(\mathbf{p},\mathbf{\alpha})}\\\\
&= p(\mathbf{x}\,|\,\mathbf{p},\mathbf{\alpha})\cdot p(\mathbf{p}\,|\,\mathbf{\alpha})\\\\
&= p(\mathbf{x}\,|\,\mathbf{p})\cdot p(\mathbf{p}\,|\,\mathbf{\alpha})\\\\
&= \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1}\\\\
&= \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{x_j + \alpha_j-1}
\end{align}
$$

This implies 
$$
\begin{align}
p(\mathbf{x}\,|\,\mathbf{\alpha}) &= \int_{\Delta_d}p(\mathbf{x},\mathbf{p}\,|\,\mathbf{\alpha})\,d\mathbf{p}\\\\
&= \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)}\int_{\Delta_d}\prod_{j=1}^d p_j^{x_j + \alpha_j-1}\,d\mathbf{p}\\\\
&= \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)}\frac{\prod_{j=1}^d \Gamma(x_j+\alpha_j)}{\Gamma(|\mathbf{x}|+|\alpha|)}
\end{align}
$$

## Q2

The log-likelihood is concave. 

In [100]:
function mymultinomial(x::Vector)
    s = 0
    result = 0
    @inbounds for i in 1:length(x)
        s += x[i]
        result += sum(log(1:s)) - sum(log(1:x[i])) - sum(log(1:(s - x[i])))
    end
    result
end

mymultinomial(training[1, :])

977.8890626874888

{T<:Any, 1}) in module Main at In[99]:2 overwritten at In[100]:2.


370.8761824169348

In [111]:
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    s = 0
    result = 0
    @inbounds for i in 1:length(x)
        s += x[i]
        result += sum(log(1:s)) - sum(log(1:x[i])) - sum(log(1:(s - x[i])))
    end

    storage = 0
    for i in 1:length(x)
        storage += lgamma(x[i] + α[i]) - lgamma(α[i])
    end
    
    return result + storage - lgamma(sum(α) + sum(x)) + lgamma(sum(α))
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
end

dirmult_logpdf

In [40]:
optdigits = readcsv("E:/Classes/BiostatisticsM280/Submissions/biostat-m280-JohnShamshoian/HW4/optdigits.tra")
training = Int64.(optdigits[:, 1:64])
digits = Int64.(optdigits[:, 65])

3823-element Array{Int64,1}:
 0
 0
 7
 4
 6
 2
 5
 5
 0
 8
 7
 1
 9
 ⋮
 1
 4
 4
 8
 9
 3
 9
 9
 4
 6
 6
 7

In [76]:
dirmult_logpdf([1;2;3],[1;1;1])

-3.3322045101752042

In [47]:
training[1, :]

64-element Array{Int64,1}:
  0
  1
  6
 15
 12
  1
  0
  0
  0
  7
 16
  6
  6
  ⋮
 15
  9
  0
  0
  0
  0
  6
 14
  7
  1
  0
  0

In [110]:
dirmult_logpdf(training', ones(Int64, 64))

3823-element Array{Float64,1}:
 -165.188
 -176.23 
 -167.774
 -165.564
 -157.79 
 -176.071
 -158.423
 -159.258
 -174.302
 -178.407
 -171.294
 -169.383
 -175.753
    ⋮    
 -160.49 
 -158.633
 -156.935
 -171.975
 -177.638
 -169.735
 -158.423
 -159.465
 -159.877
 -169.735
 -173.149
 -155.411

## Q5

